# Setup

In [111]:
# Import dependencies 
# python -m venv <name_of_virtual_environment>
# source  <name_of_virtual_environment>/bin/activate

import pandas as pd
import numpy as np
import os

In [112]:
# Build data frame with 'compras'

# set the directory containing the CSV files
directory = "/Users/annadeniz/Documents/py_scripts/digital_procurement/data/raw data/compras"

# get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# initialize an empty DataFrame to store the combined data
compras_df = pd.DataFrame()

# loop through the CSV files and append them to the combined DataFrame
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path, encoding='latin-1', delimiter=';')
    compras_df = pd.concat([compras_df, df], ignore_index=True)

# check the combined DataFrame
compras_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452617 entries, 0 to 452616
Data columns (total 24 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Número do Contrato                  452617 non-null  object 
 1   Objeto                              452407 non-null  object 
 2   Fundamento Legal                    380758 non-null  object 
 3   Modalidade Compra                   452617 non-null  object 
 4   Situação Contrato                   452617 non-null  object 
 5   Código Órgão Superior               452617 non-null  int64  
 6   Nome Órgão Superior                 452617 non-null  object 
 7   Código Órgão                        452617 non-null  int64  
 8   Nome Órgão                          452617 non-null  object 
 9   Código UG                           452617 non-null  int64  
 10  Nome UG                             452617 non-null  object 
 11  Data Assinatura Contrato  

# Data preprocessing

In [113]:
# Packages for data preprocessing

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [114]:
# Convert the text column to string type
compras_df['Objeto'] = compras_df['Objeto'].astype(str) 

In [115]:
# Tokenization
compras_df['tokens'] = compras_df['Objeto'].apply(lambda x: word_tokenize(x, language='portuguese'))

In [116]:
# Lowercasing
compras_df['tokens'] = compras_df['tokens'].apply(lambda x: [word.lower() for word in x])

In [117]:
# Stopwords Removal
stop_words = set(stopwords.words('portuguese'))

extra_stop_words = ['objeto', 'contrato', 'fornecimento', 'contratação', ':', 'serviços', 'empresa', 'execução', 'prestação', 'aquisição', 'br', 'material', 'serviço', 'especializada', 'elaboração', '.', '/', ',','!','?']
stop_words.update(extra_stop_words)

compras_df['tokens'] = compras_df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

In [118]:
# Check new dataframe with tokens

compras_df.head()

,Número do Contrato,Objeto,Fundamento Legal,Modalidade Compra,Situação Contrato,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,Código UG,...,Código Contratado,Nome Contratado,Valor Inicial Compra,Valor Final Compra,Número Licitação,Código UG Licitação,Nome UG Licitação,Código Modalidade Compra Licitação,Modalidade Compra Licitação,tokens
0,22017,"Objeto: Execução dos serviços de dedetização, ...","Fundamento Legal: Lei 10520/2002, Lei 8666/93 ...",Pregão,Publicado,40000,Ministério do Trabalho,37202,Instituto Nacional do Seguro Social,511080,...,15501741000123,P. R. DE ALMEIDA & CIA LTDA,"119988,8600","119988,8600",22016,511080.0,GERENCIA EXECUTIVA CUIABA,5.0,Pregão,"[dedetização, englobando, desinsetização, desr..."
1,32017,Objeto: Contratação de serviços de apoio oper...,Fundamento Legal: Lei nº 8.666/93,Pregão - Registro de Preço,Publicado,26000,Ministério da Educação,26403,Instituto Federal do Amazonas,158446,...,04465383000124,SUP SERVICOS DE CONSTRUCOES E MANUTENCAO EIRELI,"8240,6000","8240,6000",22015,194008.0,COORDENACAO REGIONAL DO RIO NEGRO/AM,-99.0,Pregão - Registro de Preço,"[apoio, operacional, prestados, condições, est..."
2,12017,Objeto: CONTRATAÇÃO DE SERVIÇOS GRÁFICOS PARA ...,Fundamento Legal: 10520/2002 E 8666/93,Pregão,Não se aplica,25000,Ministério da Economia,25000,Ministério da Economia - Unidades com vínculo,170010,...,17615848000128,TEIXEIRA IMPRESSAO DIGITAL E SOLUCOES GRAFICAS...,"33318,9400","33318,9400",112016,170010.0,SECRETARIA ESP. DA RECEITA FEDERAL DO BRASIL,5.0,Pregão,"[gráficos, unidades, receita, federal, brasil,..."
3,2332016,"Objeto: SERVIÇOS DE LIMPEZA, CONSERVAÇÃO E HIG...",LEI 8.666 DE 1993 E SUAS ALTERAÇÕES POSTERIORES.,Sem Informação,Fechado,32000,Ministério de Minas e Energia,91081,Empresas de Energia,910808,...,29212545000143,NOVA RIO SERVICOS GERAIS LTDA,"960348,0000","960348,0000",-2,NaN,NaN,NaN,NaN,"[limpeza, conservação, higienização, serem, re..."
4,12017,Objeto: Contratação de empresa especializada n...,"Fundamento Legal: lei 8666/93, artg 24 - ii",Dispensa de Licitação,Publicado,25000,Ministério da Economia,25000,Ministério da Economia - Unidades com vínculo,170258,...,07346326000114,REPROS SOLUCOES EM DOCUMENTOS LTDA,"7200,0000","7200,0000",12017,170258.0,INSPETORIA DA RFB NO RIO DE JANEIRO,6.0,Dispensa de Licitação,"[impressao, equipamentos, manutenção, preventi..."


In [119]:
# Vectorization
vectorizer = TfidfVectorizer(max_df=0.9, min_df=100)
X = vectorizer.fit_transform(compras_df['tokens'].apply(lambda x: ' '.join(x)))

# Topic modelling

In [120]:
# Perform topic modelling using Latent Dirichlet Allocation (LDA)
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=100, random_state=42)
lda.fit(X)

LatentDirichletAllocation(n_components=100, random_state=42)

In [121]:
# Get words and corresponding weights resulting from the topic model in a table

def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    
    return pd.DataFrame(topic_dict)

In [122]:
# Get name of each title: augment the number of words per topic and base it on the results 

no_top_words = 15

display_topics(lda, vectorizer.get_feature_names_out(), no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,...,Topic 95 words,Topic 95 weights,Topic 96 words,Topic 96 weights,Topic 97 words,Topic 97 weights,Topic 98 words,Topic 98 weights,Topic 99 words,Topic 99 weights
0,água,1244.5,reabilitação,842.3,mão,2508.4,receita,858.2,instalação,474.6,...,bens,668.8,ml,370.5,tensão,397.3,meses,1217.0,assinatura,408.0
1,esgoto,1193.6,próteses,526.8,obra,2393.5,brasil,749.2,montagem,434.1,...,imóveis,662.8,lavagem,256.2,alta,296.9,12,1065.3,descrito,277.6
2,laboratorial,942.6,órteses,470.5,exclusiva,2037.4,federal,556.3,divisórias,318.3,...,gráficos,530.6,patrocínio,240.4,subestação,237.9,período,749.0,primeira,275.2
3,univasf,789.9,profissional,429.8,dedicação,1948.8,delegacia,510.9,desmontagem,209.0,...,manutenção,346.4,realização,235.6,baixa,206.9,doze,635.9,ifpr,258.9
4,coleta,602.2,medida,417.9,continuados,1208.9,chaveiro,222.7,goiânia,205.3,...,móveis,339.2,mesa,231.0,média,196.2,60,230.9,cláusula,232.7
5,uso,551.8,favor,373.8,disponibilização,1136.3,dtcea,222.0,persianas,203.2,...,áreas,326.3,roupas,228.5,ufrgs,185.4,prazo,222.6,anual,187.5
6,tratamento,461.9,paciente,343.3,regime,1088.0,monitoramento,216.5,ifg,201.9,...,externas,240.6,evento,213.7,corpo,162.7,periodo,182.6,digital,156.6
7,contidas,452.7,segurados,340.0,conservação,553.0,cftv,196.7,contrataçao,175.2,...,internas,236.6,cultural,195.3,nobreak,120.1,anatel,154.8,revista,150.9
8,características,445.6,prontuário,289.1,limpeza,548.7,sistema,170.0,cabeamento,163.0,...,ifce,215.8,bndes,187.7,subestações,119.5,2019,140.9,conforme,150.3
9,demais,363.4,sob,288.8,prestados,520.9,alarme,168.8,go,162.2,...,conjunto,199.8,oficina,160.0,entrada,114.9,sessenta,138.8,líquido,142.4


In [123]:
# Create df with topics per item

W = lda.transform(X)

df_w = pd.DataFrame(W)

df_w.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.001736,0.001736,0.001736,0.001736,0.001736,0.001736,0.001736,0.001736,0.138207,0.001736,...,0.001736,0.001736,0.001736,0.001736,0.118934,0.001736,0.001736,0.001736,0.001736,0.001736
1,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,...,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761
2,0.001803,0.001803,0.001803,0.291804,0.001803,0.001803,0.001803,0.071687,0.001803,0.001803,...,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803
3,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,...,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439
4,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.564944,0.002396,0.002396,...,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396


In [124]:
# Merge compras_df with df with topics per item

compras_topics_df = pd.concat([compras_df,df_w], axis=1)

compras_topics_df.head()

,Número do Contrato,Objeto,Fundamento Legal,Modalidade Compra,Situação Contrato,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,Código UG,...,90,91,92,93,94,95,96,97,98,99
0,22017,"Objeto: Execução dos serviços de dedetização, ...","Fundamento Legal: Lei 10520/2002, Lei 8666/93 ...",Pregão,Publicado,40000,Ministério do Trabalho,37202,Instituto Nacional do Seguro Social,511080,...,0.001736,0.001736,0.001736,0.001736,0.118934,0.001736,0.001736,0.001736,0.001736,0.001736
1,32017,Objeto: Contratação de serviços de apoio oper...,Fundamento Legal: Lei nº 8.666/93,Pregão - Registro de Preço,Publicado,26000,Ministério da Educação,26403,Instituto Federal do Amazonas,158446,...,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761,0.002761
2,12017,Objeto: CONTRATAÇÃO DE SERVIÇOS GRÁFICOS PARA ...,Fundamento Legal: 10520/2002 E 8666/93,Pregão,Não se aplica,25000,Ministério da Economia,25000,Ministério da Economia - Unidades com vínculo,170010,...,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803,0.001803
3,2332016,"Objeto: SERVIÇOS DE LIMPEZA, CONSERVAÇÃO E HIG...",LEI 8.666 DE 1993 E SUAS ALTERAÇÕES POSTERIORES.,Sem Informação,Fechado,32000,Ministério de Minas e Energia,91081,Empresas de Energia,910808,...,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439,0.002439
4,12017,Objeto: Contratação de empresa especializada n...,"Fundamento Legal: lei 8666/93, artg 24 - ii",Dispensa de Licitação,Publicado,25000,Ministério da Economia,25000,Ministério da Economia - Unidades com vínculo,170258,...,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396,0.002396


# Set initial dataset for labelling

In [125]:
# Analyse topics with potential for digital technology

topics_df = pd.DataFrame(display_topics(lda, vectorizer.get_feature_names_out(), no_top_words))

topics_df.to_csv('topics_df.csv', index=False)

In [126]:
# Topics with potential for gitial technology

digital_topics = topics_df.loc[:,['Topic 36 words', 'Topic 64 words', 'Topic 75 words', 'Topic 92 words']]

digital_topics

,Topic 36 words,Topic 64 words,Topic 75 words,Topic 92 words
0,solução,digitais,processos,software
1,garantia,dados,secretaria,suporte
2,suporte,processamento,direitos,licenças
3,instalação,pacote,tic,técnico
4,rede,certificados,tecnologia,atualização
5,site,meio,informação,licença
6,configuração,correios,técnicos,manutenção
7,on,emissão,ministério,uso
8,armazenamento,produtos,desenvolvimento,softwares
9,meses,digital,gestão,sistema


In [127]:
# Create subset of the dataset with most relevant data for labelling

subset_1 = compras_topics_df[compras_topics_df[92] > 0.75] # 732 entries

subset_2 = compras_topics_df[compras_topics_df[92] < 0.75]

subset_2 = subset_2.sample(frac=0.0008, random_state=42) # 362 entries

subset_1.info()
subset_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 732 entries, 697 to 451686
Columns: 125 entries, Número do Contrato to 99
dtypes: float64(102), int64(4), object(19)
memory usage: 720.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 362 entries, 371484 to 148642
Columns: 125 entries, Número do Contrato to 99
dtypes: float64(102), int64(4), object(19)
memory usage: 356.3+ KB


In [128]:
# Join datasets

initial_df = pd.concat([subset_1,subset_2])

initial_df = initial_df.sample(frac=1)

initial_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 354253 to 118611
Columns: 125 entries, Número do Contrato to 99
dtypes: float64(102), int64(4), object(19)
memory usage: 1.1+ MB


# Label data

In [129]:
# Export dataset for labeling

initial_df.to_csv('initial_df.csv')

In [130]:
# Launch Open Source Data Labeling Platform - https://labelstud.io/

In [131]:
# Import labeled data 

# Defined categories: 'Digital Technology' and 'Other'

seed_df = pd.read_json('labeled_data.json')

seed_df.head()

,Unnamed: 0,Número do Contrato,Objeto,Fundamento Legal,Modalidade Compra,Situação Contrato,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,...,97,98,99,id,sentiment,annotator,annotation_id,created_at,updated_at,lead_time
0,25564,1122018,Objeto: PRESTAÇÃO DE SERVIÇO DE PACOTE DE ACES...,Fundamento Legal: LEI 8666/93,Inexigibilidade de Licitação,Não se aplica,26000,Ministério da Educação,26247,Universidade Federal de Santa Maria,...,0.003679,0.003679,0.003679,1362,Other,1,48,2023-03-22 17:25:52.126185+00:00,2023-03-22 17:25:52.126224+00:00,39.786
1,307257,352021,Objeto: O OBJETO DO PRESENTE INSTRUMENTO É A C...,,Inexigibilidade de Licitação,Não se aplica,36000,Ministério da Saúde,36201,Fundação Oswaldo Cruz,...,0.002161,0.002161,0.002161,1363,Other,1,49,2023-03-22 17:26:16.896686+00:00,2023-03-22 17:26:16.896715+00:00,24.531
2,86384,22015,Objeto: PRESTAÇÃO DE SERVIÇOS DE LOCAÇÃO DE CE...,Fundamento Legal: LEI 8.666/93,Pregão - Registro de Preço,Não se aplica,25000,Ministério da Economia,25000,Ministério da Economia - Unidades com vínculo,...,0.002157,0.002157,0.002157,1364,Other,1,50,2023-03-22 17:26:24.304412+00:00,2023-03-22 17:26:24.304443+00:00,7.204
3,133339,102018,Objeto: Contrato de prestação de serviços de s...,Fundamento Legal: LEI 8.666/93 E DEMAIS CORREL...,Pregão,Publicado,22000,"Ministério da Agricultura, Pecuária e Abastec",22202,Empresa Brasileira de Pesquisa Agropecuária,...,0.002524,0.002524,0.002524,1365,Digital Technology,1,51,2023-03-22 17:28:00.333478+00:00,2023-03-22 17:28:00.333501+00:00,95.832
4,444569,262013,Objeto: Aquisição de licenças perpétuas de sol...,"Fundamento Legal: Leis 8666/93, 10520/02 e Dec...",Pregão,Publicado,25000,Ministério da Economia,25000,Ministério da Economia - Unidades com vínculo,...,0.001630,0.001630,0.001630,1366,Digital Technology,1,52,2023-03-22 17:28:12.296439+00:00,2023-03-22 17:28:12.296489+00:00,11.782


# Model

In [132]:
# Import packages

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [133]:
# Set features and target variables

X_seed = seed_df['tokens']
y = seed_df['sentiment']

In [134]:
# Split the data into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X_seed, y, test_size=0.25, random_state=42)

In [135]:
# Vectorize the text using TF-IDF vectorizer

X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [136]:
# Train a logistic regression model

classifier = LogisticRegression()
classifier.fit(X_train_vec, y_train)

LogisticRegression()

In [137]:
# Predict the labels for the test set

y_pred = classifier.predict(X_test_vec)

In [138]:
# Evaluate the accuracy of the model

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.96


In [139]:
# Print the classification report

report = classification_report(y_test, y_pred, zero_division=0)
print(report)

                    precision    recall  f1-score   support

Digital Technology       0.97      0.95      0.96       148
             Other       0.94      0.97      0.95       126

          accuracy                           0.96       274
         macro avg       0.96      0.96      0.96       274
      weighted avg       0.96      0.96      0.96       274



In [140]:
# Save trained model

import pickle

with open('trained_classifier.pkl', 'wb') as f:
    pickle.dump(classifier, f)

# Predicting labels for the whole dataset

In [141]:
# Vectorize the text using the same vectorizer as before

X_unlabeled_vec = X

In [142]:
# Predict the labels for the unlabeled data

y_pred = classifier.predict(X_unlabeled_vec)

In [143]:
# Store the predicted labels in original dataframe

compras_df['category'] = y_pred

In [144]:
# Check final distribution of categories in the dataset

categories = compras_df.groupby('category')

categories['Objeto'].count()

category
Digital Technology      9740
Other                 442877
Name: Objeto, dtype: int64

In [145]:
# Check results 

compras_df_dig = compras_df[compras_df['category'] == 'Digital Technology']

compras_df_dig['Objeto']

27        Objeto: PRESTAÇÃO DE SERVIÇOS DE LICENÇA DE US...
46        Objeto: Renovação de licenças de software de s...
101       Objeto: Contratação de Solução de Sistema de p...
159       OBJETO: Prestação de subscrição suse e da pres...
246       Objeto: PRESTAÇÃO DE SERVIÇOS DE APOIO TÉCNICO...
                                ...                        
452531    Objeto: CONTRATAÇÃO DE SERVIÇOS DE TECNOLOGIA ...
452540    Objeto: O PRESENTE CONTRATO TEM POR OBJETO A C...
452591    Objeto: CONTRATAÇÃO DE SOLUÇÃO DE SOFTWARE PAR...
452601    Objeto: AQUISIÇÃO DE SOLUÇÃO DE COMUNICAÇÃO DE...
452608    Objeto: AQUISIÇÃO DE LICENÇAS DE SOFTWARE DO T...
Name: Objeto, Length: 9740, dtype: object

# Descriptive analysis with new labels

In [146]:
compras_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452617 entries, 0 to 452616
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Número do Contrato                  452617 non-null  object 
 1   Objeto                              452617 non-null  object 
 2   Fundamento Legal                    380758 non-null  object 
 3   Modalidade Compra                   452617 non-null  object 
 4   Situação Contrato                   452617 non-null  object 
 5   Código Órgão Superior               452617 non-null  int64  
 6   Nome Órgão Superior                 452617 non-null  object 
 7   Código Órgão                        452617 non-null  int64  
 8   Nome Órgão                          452617 non-null  object 
 9   Código UG                           452617 non-null  int64  
 10  Nome UG                             452617 non-null  object 
 11  Data Assinatura Contrato  

In [147]:
# Distribution of contracts per category per year

compras_df['Data Publicação DOU'] = pd.to_datetime(compras_df['Data Publicação DOU'], format='%d/%m/%Y')

compras_df['Ano Contrato'] = compras_df['Data Publicação DOU'].dt.year

category_year = pd.DataFrame(compras_df.groupby('Ano Contrato')['category'].value_counts().unstack().fillna(0).astype(int))

category_year

category,Digital Technology,Other
Ano Contrato,,
2013,962,45236
2014,1008,50447
2015,786,41637
2016,919,43120
2017,1093,44088
2018,1243,48570
2019,988,49482
2020,1209,52512
2021,863,37264


In [148]:
# Distribution of contracts per category per department

category_department = pd.DataFrame(compras_df.groupby('Nome Órgão Superior')['category'].value_counts().unstack().fillna(0).astype(int))

category_department

category,Digital Technology,Other
Nome Órgão Superior,,
Advocacia-Geral da União,47,2226
Banco Central do Brasil - Orçamento Fiscal e,235,6056
Controladoria-Geral da União,70,452
Justiça Federal,98,358
Justiça do Trabalho,26,134
"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",4,129
MINISTERIO DA ECONOMIA,3,810
MINISTERIO DA SAUDE,15,194
MINISTERIO DAS COMUNICACOES,33,706


In [149]:
# Distribution of contracts per department per year

department_year = pd.DataFrame(compras_df.groupby('Nome Órgão Superior')['Ano Contrato'].value_counts().unstack().fillna(0).astype(int))

department_year

Ano Contrato,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Nome Órgão Superior,,,,,,,,,,
Advocacia-Geral da União,141,228,147,160,319,307,303,309,157,202
Banco Central do Brasil - Orçamento Fiscal e,1053,1021,736,862,741,627,436,361,277,177
Controladoria-Geral da União,34,60,38,46,58,70,53,77,66,20
Justiça Federal,70,69,64,48,51,64,53,37,0,0
Justiça do Trabalho,28,29,24,32,35,11,1,0,0,0
"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",21,19,24,26,18,14,11,0,0,0
MINISTERIO DA ECONOMIA,10,803,0,0,0,0,0,0,0,0
MINISTERIO DA SAUDE,35,49,34,21,35,30,5,0,0,0
MINISTERIO DAS COMUNICACOES,124,90,103,166,143,61,35,17,0,0


In [150]:
# Get the unique values from the column 'Nome Órgão Superior'

org_sup_values = compras_df['Nome Órgão Superior'].value_counts().to_dict()

print(org_sup_values)

{'Ministério da Educação': 134470, 'Ministério da Defesa': 98768, 'Ministério do Trabalho': 33568, 'Ministério da Saúde': 32471, 'Ministério da Economia': 29909, 'Ministério de Minas e Energia': 22451, 'Ministério da Justiça e Segurança Pública': 21896, 'Ministério da Infraestrutura': 12177, 'Ministério da Agricultura, Pecuária e Abastec': 11536, 'Ministério do Desenvolvimento Regional': 9616, 'Banco Central do Brasil - Orçamento Fiscal e ': 6291, 'Ministério da Ciência, Tecnologia, Inovações ': 6193, 'Ministério do Turismo': 5663, 'Ministério do Meio Ambiente': 5211, 'MINISTERIO DE MINAS E ENERGIA': 3981, 'Ministério das Comunicações': 3167, 'Presidência da República': 2823, 'Advocacia-Geral da União': 2273, 'Ministério da Cidadania': 2227, 'PRESIDENCIA DA REPUBLICA': 1777, 'Ministério das Relações Exteriores': 894, 'MINISTERIO DA ECONOMIA': 813, 'MINISTERIO DAS COMUNICACOES': 739, 'Ministério do Trabalho e Emprego': 719, 'Ministério do Desenvolvimento Agrário': 546, 'Ministério da Mu

In [151]:
# Merge departments together

departments_group = {'Ministério da Educação': 'Ministério da Educação', 
                     'Ministério da Defesa': 'Ministério da Defesa', 
                     'Ministério do Trabalho': 'Ministério do Trabalho', 
                     'Ministério da Saúde': 'Ministério da Saúde', 
                     'Ministério da Economia': 'Ministério da Economia', 
                     'Ministério de Minas e Energia': 'Ministério de Minas e Energia', 
                     'Ministério da Justiça e Segurança Pública': 'Ministério da Justiça e Segurança Pública', 
                     'Ministério da Infraestrutura': 'Ministério da Infraestrutura', 
                     'Ministério da Agricultura, Pecuária e Abastec': 'Ministério da Agricultura, Pecuária e Abastecimento', 
                     'Ministério do Desenvolvimento Regional': 'Ministério do Desenvolvimento Regional', 
                     'Banco Central do Brasil - Orçamento Fiscal e ': 'Banco Central do Brasil', 
                     'Ministério da Ciência, Tecnologia, Inovações ': 'Ministério da Ciência, Tecnologia e Inovações', 
                     'Ministério do Turismo': 'Ministério do Turismo', 
                     'Ministério do Meio Ambiente': 'Ministério do Meio Ambiente', 
                     'MINISTERIO DE MINAS E ENERGIA': 'Ministério de Minas e Energia', 
                     'Ministério das Comunicações': 'Ministério das Comunicações', 
                     'Presidência da República': 'Presidência da República', 
                     'Advocacia-Geral da União': 'Ministério da Justiça e Segurança Pública', 
                     'Ministério da Cidadania': 'Ministério da Cidadania', 
                     'PRESIDENCIA DA REPUBLICA': 'Presidência da República', 
                     'Ministério das Relações Exteriores': 'Ministério das Relações Exteriores', 
                     'MINISTERIO DA ECONOMIA': 'Ministério da Economia', 
                     'MINISTERIO DAS COMUNICACOES': 'Ministério das Comunicações', 
                     'Ministério do Trabalho e Emprego': 'Ministério do Trabalho', 
                     'Ministério do Desenvolvimento Agrário': 'Ministério da Agricultura, Pecuária e Abastecimento', 
                     'Ministério da Mulher, Família e Direitos Huma': 'Ministério da Cidadania', 
                     'Controladoria-Geral da União': 'Ministério da Justiça e Segurança Pública', 
                     'Justiça Federal': 'Ministério da Justiça e Segurança Pública', 
                     'MINISTERIO DA SAUDE': 'Ministério da Saúde', 
                     'Ministério da Pesca e Aquicultura': 'Ministério da Agricultura, Pecuária e Abastecimento', 
                     'Justiça do Trabalho': 'Ministério da Justiça e Segurança Pública', 
                     'MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO': 'Ministério da Agricultura, Pecuária e Abastecimento', 
                     'Ministério da Previdência Social': 'Ministério da Economia', 
                     'Ministério do Planejamento, Desenvolvimento e': 'Ministério da Economia', 
                     'MINISTERIO DOS TRANSPORTES': 'Ministério da Infraestrutura', 
                     'MINISTERIO DO PLANEJAMENTO,DESENV. E GESTÃO': 'Ministério da Economia', 
                     'Ministério das Mulheres, Igualdade Racial, da': 'Ministério da Cidadania'}

compras_df['Ministério'] = compras_df['Nome Órgão Superior'].replace(departments_group)

In [152]:
# Distribution of contracts of digital technology per department per year

digi_department_year = compras_df[compras_df['category'] == 'Digital Technology']

digi_department_year = pd.DataFrame(digi_department_year.groupby('Ministério')['Ano Contrato'].value_counts().unstack().fillna(0).astype(int))

digi_department_year

Ano Contrato,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Ministério,,,,,,,,,,
Banco Central do Brasil,32,37,19,20,37,21,12,25,22,10
"Ministério da Agricultura, Pecuária e Abastecimento",42,35,36,33,25,47,21,24,16,11
Ministério da Cidadania,7,19,6,11,9,23,9,12,11,1
"Ministério da Ciência, Tecnologia e Inovações",38,44,28,36,41,51,41,37,41,19
Ministério da Defesa,93,98,96,121,118,130,125,195,125,100
Ministério da Economia,116,142,120,144,142,122,106,146,49,43
Ministério da Educação,210,224,151,222,243,347,303,377,307,287
Ministério da Infraestrutura,43,40,43,35,71,51,35,51,43,20
Ministério da Justiça e Segurança Pública,50,64,47,45,66,90,53,77,49,24


In [153]:
# Distribution of percentage of contracts of digital technology per department per year

perc_digi_department_year = compras_df.groupby(['Ministério', 'Ano Contrato'])['category'].apply(lambda x: (x == 'Digital Technology').sum() / len(x) * 100)

perc_digi_department_year = pd.DataFrame(perc_digi_department_year).reset_index()

perc_digi_department_year = perc_digi_department_year.pivot(index='Ministério', columns='Ano Contrato', values='category').fillna(0)

perc_digi_department_year = perc_digi_department_year.round(2)

perc_digi_department_year

Ano Contrato,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Ministério,,,,,,,,,,
Banco Central do Brasil,3.04,3.62,2.58,2.32,4.99,3.35,2.75,6.93,7.94,5.65
"Ministério da Agricultura, Pecuária e Abastecimento",2.25,1.92,2.53,2.71,2.07,3.74,1.82,1.90,2.40,1.99
Ministério da Cidadania,4.67,7.22,4.44,7.86,5.81,4.61,1.47,3.35,3.63,0.70
"Ministério da Ciência, Tecnologia e Inovações",5.69,5.39,4.95,6.46,6.74,7.35,6.07,5.27,7.59,5.19
Ministério da Defesa,1.53,1.23,1.47,1.48,1.34,1.27,0.99,1.28,0.96,1.00
Ministério da Economia,3.34,3.19,3.46,4.21,4.34,3.83,3.32,4.51,3.03,2.68
Ministério da Educação,1.64,1.62,1.21,1.76,1.92,2.33,1.94,2.19,2.60,2.71
Ministério da Infraestrutura,3.52,3.02,3.43,3.20,5.78,3.43,2.63,3.31,4.30,2.79
Ministério da Justiça e Segurança Pública,2.32,2.82,2.32,2.09,2.86,2.77,1.48,2.23,2.01,1.44


# Performance metric across departments

In [154]:
# Distribution of percentage of contracts of digital technology per department

perc_digi_department = compras_df.groupby(['Ministério'])['category'].apply(lambda x: (x == 'Digital Technology').sum() / len(x) * 100)

perc_digi_department = pd.DataFrame(perc_digi_department)

perc_digi_department = perc_digi_department.sort_values(by='category', ascending=False)

perc_digi_department = perc_digi_department.round(2)

perc_digi_department = perc_digi_department.rename(columns={'category': 'Digital Technology %'})

perc_digi_department

,Digital Technology %
Ministério,
"Ministério da Ciência, Tecnologia e Inovações",6.07
Ministério das Relações Exteriores,5.93
Presidência da República,4.87
Ministério do Turismo,4.22
Ministério da Cidadania,3.92
Banco Central do Brasil,3.74
Ministério das Comunicações,3.71
Ministério da Economia,3.65
Ministério da Infraestrutura,3.54


In [155]:
# create a new dataframe grouped by department
#digi_dept = compras_df[compras_df['category'] == 'Digital Technology'].groupby('Código Órgão Superior')

# calculate the percentage of digital technology contracts per department
#digi_pct = digi_dept['Valor Final Compra'].count() / compras_df.groupby('Código Órgão Superior')['Valor Final Compra'].count() * 100

# calculate the average spending on digital technology contracts per department
#digi_avg_spending = digi_dept['Valor Final Compra'].sum() / digi_dept['Valor Final Compra'].count()

# create a new dataframe combining the two metrics
#digi_metrics = pd.DataFrame({'digital_technology_pct': digi_pct, 'digital_technology_avg_spending': digi_avg_spending})

# merge the new dataframe with the original one to add department names
#digi_metrics = pd.merge(digi_metrics, compras_df[['Código Órgão Superior', 'Nome Órgão Superior']].drop_duplicates(), on='Código Órgão Superior')

# sort the dataframe by descending percentage of digital technology contracts
#digi_metrics = digi_metrics.sort_values('digital_technology_pct', ascending=False)

# reset the index and rename columns for readability
#digi_metrics = digi_metrics.reset_index(drop=True)
#digi_metrics.columns = ['Department Code', 'Digital Technology Contract Percentage', 'Digital Technology Average Spending', 'Department Name']

# display the resulting dataframe
#digi_metrics

In [157]:
compras_df['Valor Contrato'] = compras_df['Valor Final Compra'].str.replace(',', '.').astype(float)

# Calculate Digital Technology Contract Percentage per department
digital_percentage = (compras_df['category'] == 'Digital Technology').groupby(compras_df['Ministério']).mean() * 100

# Calculate Digital Technology Average Spending per department
digital_spending = compras_df[compras_df['category'] == 'Digital Technology'].groupby('Ministério')['Valor Contrato'].mean()

# Calculate the weighted average of the two metrics
w1 = 0.7
w2 = 0.3
digital_indicator = (digital_percentage * w1) + (digital_spending * w2)

# Sort the indicator values in descending order
digital_indicator = digital_indicator.sort_values(ascending=False)

digital_indicator = digital_indicator.round(2)

# Print the results
digital_indicator


Ministério
Ministério do Trabalho                                 7825658.21
Ministério da Economia                                 1667697.02
Ministério da Cidadania                                1664577.58
Ministério da Infraestrutura                           1623582.20
Ministério da Saúde                                    1244665.58
Ministério das Relações Exteriores                     1176311.39
Ministério da Justiça e Segurança Pública              1098216.23
Banco Central do Brasil                                 723118.03
Ministério da Agricultura, Pecuária e Abastecimento     698530.98
Ministério da Educação                                  678316.61
Ministério do Meio Ambiente                             671984.48
Ministério das Comunicações                             595068.26
Ministério da Defesa                                    558969.59
Ministério do Desenvolvimento Regional                  505727.14
Presidência da República                                458576.03

In [158]:
from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()

# fit and transform the indicator values
indicator_values = digital_indicator.values.reshape(-1, 1)
normalized_values = scaler.fit_transform(indicator_values)

# add the normalized values to the dataframe
normalized_values


array([[1.        ],
       [0.18198445],
       [0.18157007],
       [0.1761243 ],
       [0.12578951],
       [0.11670943],
       [0.10633537],
       [0.05650781],
       [0.0532417 ],
       [0.05055645],
       [0.0497153 ],
       [0.03949785],
       [0.03470255],
       [0.02762989],
       [0.0213664 ],
       [0.0174862 ],
       [0.01517548],
       [0.        ]])

In [ ]:
# calculate the performance indicator
digi_spending_per_dept = compras_df[compras_df['category'] == 'Digital Technology'].groupby('Ministério')['Valor Contrato'].agg(['sum', 'count'])
digi_spending_per_dept['Digital Technology Average Spending'] = digi_spending_per_dept['sum'] / digi_spending_per_dept['count']
digi_spending_per_dept.drop(['sum', 'count'], axis=1, inplace=True)
digi_contract_pct_per_dept = compras_df.groupby('Código Órgão Superior')['category'].apply(lambda x: (x == 'Digital Technology').mean() * 100)
performance_indicator = pd.concat([digi_contract_pct_per_dept, digi_spending_per_dept], axis=1)
performance_indicator = performance_indicator.fillna(0)

# normalize the performance indicator
performance_indicator_norm = (performance_indicator - performance_indicator.min()) / (performance_indicator.max() - performance_indicator.min())

# create the desired dataframe
performance_df = pd.DataFrame({'Ministério': performance_indicator_norm.index, 'Performance Indicator': performance_indicator_norm.values.flatten()})
performance_df.set_index('Ministério', inplace=True)
